# SmartStay

## Imports

In [1]:
import json
import csv
import os

import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

from typing import Dict, Tuple
from IPython.display import display

## Data Preprocessing

### Raw text data to csv conversion

In [2]:
import json
import csv

def json_to_csv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['hotel_url', 'author', 'date', 'rating', 'title', 'text', 'property_dict']
        csv_writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        csv_writer.writeheader()

        for line in infile:
            data = json.loads(line)
            
            # Ensure all fields are converted to strings to avoid encoding issues
            for key in data.keys():
                if isinstance(data[key], dict):
                    data[key] = json.dumps(data[key], ensure_ascii=False)  # Serialize dictionaries as JSON
                elif data[key] is None:
                    data[key] = ''  # Handle None values
                else:
                    data[key] = str(data[key])  # Convert other types to string
            
            csv_writer.writerow(data)

input_file = '../data/HotelRec.txt'
output_file = '../data/hotel_reviews.csv'
json_to_csv(input_file, output_file)

### Encoding Hotel and Authors

In [3]:
def create_label_encoders(df: pd.DataFrame) -> Tuple[Dict[str, int], Dict[str, int]]:
    """
    Create label encoders (mapping dictionaries) for hotel_url and author columns
    
    Args:
        df (pd.DataFrame): Input dataframe containing hotel_url and author columns
        
    Returns:
        Tuple[Dict[str, int], Dict[str, int]]: Hotel URL and author mapping dictionaries
    """
    # mapping dictionaries
    hotel_mapping = {url: idx for idx, url in enumerate(df['hotel_url'].unique())}
    author_mapping = {author: idx for idx, author in enumerate(df['author'].unique())}
    
    return hotel_mapping, author_mapping

def encode_hotel_author(df: pd.DataFrame) -> pd.DataFrame:
    """
    Encode hotel_url and author columns to integer values
    
    Args:
        df (pd.DataFrame): Input dataframe containing hotel_url and author columns
        
    Returns:
        pd.DataFrame: DataFrame with encoded columns added
    """
    # copy to avoid modifying the original dataframe
    df_encoded = df.copy()
    
    # mapping dictionaries
    hotel_mapping, author_mapping = create_label_encoders(df)
    
    # encoded columns
    df_encoded['hotel_id'] = df_encoded['hotel_url'].map(hotel_mapping)
    df_encoded['author_id'] = df_encoded['author'].map(author_mapping)
    
    # Print statistics
    print(f"Number of unique hotels: {len(hotel_mapping)}")
    print(f"Number of unique authors: {len(author_mapping)}")
    
    return df_encoded, hotel_mapping, author_mapping

In [4]:
# Encode the data , running the functions defined above

df = dd.read_csv('../data/hotel_reviews.csv')
df_encoded, hotel_mapping, author_mapping = encode_hotel_author(df)

# df_encoded now contains new columns 'hotel_id' and 'author_id'
# the original columns are preserved

Number of unique hotels: 365057
Number of unique authors: 21891404


### Data filter and save

In [5]:
# remove hotel_url and author columns, as encoded data is now there
df_encoded = df_encoded.drop(['hotel_url', 'author'], axis=1)
# remove empty rating rows
df_encoded = df_encoded[df_encoded['rating'].notnull()]

In [6]:
## filter authors with 20+ reviews only [20-core]

# enable progress bar
pbar = ProgressBar()
pbar.register()

# partition optimized to 25, using 64GB DDR5 6000MT/s memory 
print("Computing npartitions...")
npartitions = 25
df_encoded = df_encoded.repartition(npartitions=npartitions)

# compute author counts using optimized aggregation
print("Computing author frequencies...")
author_counts = df_encoded.author_id.value_counts(split_every=8).compute()
print("Computing frequent authors...")
frequent_authors = set(author_counts[author_counts >= 20].index)
print(f"Found {len(frequent_authors)} frequent authors")

# filter with optimized settings
print("Filtering and saving data...")
df_frequent = df_encoded[df_encoded.author_id.isin(frequent_authors)]

Computing npartitions...
Computing author frequencies...
[########################################] | 100% Completed | 3hr 29m
Computing frequent authors...
Found 189992 frequent authors
Filtering and saving data...


In [7]:
# save filtered data to filtered_reviews.csv
# using chunks to avoid exponential memory consumption
chunk_size = 50000

# export the filtered DataFrame to CSV in chunks
# this saves the data as .part files insie the folder
df_frequent.to_csv('../data/filtered_reviews.csv', index=False, chunksize=chunk_size)

[########################################] | 100% Completed | 3hr 35m


['C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\00.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\01.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\02.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\03.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\04.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\05.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\06.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\07.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\08.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\data\\filtered_reviews.csv\\09.part',
 'C:\\Users\\cyclo\\

In [9]:
# data is saved seprately in parts and combined these parts for single filtered reviews

folder_path = '../data/filtered_reviews.csv'  # folder containing the .part files

# output filtered CSV file
output_file = '../data/combined_filtered_reviews.csv'

# open the output file in write mode with UTF-8 encoding
with open(output_file, 'w', encoding='utf-8') as outfile:
    first_file = True  # to track the first file for including headers

    # Iterate through all .part files in the folder (sorted)
    for part_file in sorted(os.listdir(folder_path)):
        if part_file.endswith('.part'):  # only process .part files (redundant but safe)
            with open(os.path.join(folder_path, part_file), 'r', encoding='utf-8') as infile:
                if first_file:
                    # write the header from the first file
                    outfile.write(infile.read())
                    first_file = False
                else:
                    # skiped the header for subsequent files
                    next(infile)  # skipped the first line
                    outfile.write(infile.read())

## Data Exploration

In [10]:
# load CSV file
file_path = "../data/combined_filtered_reviews.csv"
df = pd.read_csv(file_path)

# Display basic information
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7379698 entries, 0 to 7379697
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   date           object 
 1   rating         float64
 2   title          object 
 3   text           object 
 4   property_dict  object 
 5   hotel_id       int64  
 6   author_id      int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 394.1+ MB
None
                  date  rating  \
0  2017-03-01T00:00:00     5.0   
1  2016-01-01T00:00:00     5.0   
2  2015-07-01T00:00:00     5.0   
3  2017-05-01T00:00:00     4.0   
4  2017-01-01T00:00:00     5.0   

                                               title  \
0                                     Natural Luxury   
1                 Loved out stay, a lot of character   
2  Really nice staff, yummy breakfast and the mon...   
3  Gorgeous agriturismo with beautiful grounds an...   
4                      Really enjoyed the experience   

                         

In [11]:
def analyze_author_ratings(df):
    # Count number of reviews by each author
    author_review_counts = df.groupby('author_id').size().reset_index(name='review_count')
    
    # Count unique hotels rated by each author
    author_hotel_counts = df.groupby('author_id')['hotel_id'].nunique().reset_index(name='hotels_rated')
    
    # Combine the information
    author_stats = author_review_counts.merge(author_hotel_counts, on='author_id')
    
    # Find authors with most and least hotel ratings
    most_active = author_stats.nlargest(5, 'hotels_rated')
    least_active = author_stats.nsmallest(5, 'hotels_rated')
    
    # Calculate average reviews per hotel for each author
    author_stats['avg_reviews_per_hotel'] = author_stats['review_count'] / author_stats['hotels_rated']
    
    # Basic statistics
    summary_stats = {
        'total_authors': len(author_stats),
        'avg_hotels_per_author': author_stats['hotels_rated'].mean(),
        'median_hotels_per_author': author_stats['hotels_rated'].median()
    }
    
    return most_active, least_active, summary_stats

# Analyze the data
most_active_authors, least_active_authors, stats = analyze_author_ratings(df)

print("\nMost active authors (by number of hotels rated):")
print(most_active_authors)

print("\nLeast active authors (by number of hotels rated):")
print(least_active_authors)

print("\nSummary Statistics:")
print(f"Total number of unique authors: {stats['total_authors']}")
print(f"Average hotels rated per author: {stats['avg_hotels_per_author']:.2f}")
print(f"Median hotels rated per author: {stats['median_hotels_per_author']:.2f}")


Most active authors (by number of hotels rated):
       author_id  review_count  hotels_rated
44           202       1193017         38828
252         1504         23889         18894
74           338           612           580
22365     194488           443           412
3620       27812           536           387

Least active authors (by number of hotels rated):
        author_id  review_count  hotels_rated
189987   19861831            22             1
189989   20660091            22             1
189990   20862840            20             1
189991   21352964            20             1
188727    6925116            23             2

Summary Statistics:
Total number of unique authors: 189992
Average hotels rated per author: 31.13
Median hotels rated per author: 26.00


In [12]:
def get_author_reviews(df, author_id):
    # Filter reviews for the specific author
    author_reviews = df[df['author_id'] == author_id].copy()
    
    # Sort by date to see reviews chronologically
    author_reviews['date'] = pd.to_datetime(author_reviews['date'])
    author_reviews = author_reviews.sort_values('date')
    
    # Add some summary statistics
    summary = {
        'total_reviews': len(author_reviews),
        'unique_hotels': author_reviews['hotel_id'].nunique(),
        'average_rating': author_reviews['rating'].mean(),
        'date_range': f"From {author_reviews['date'].min().date()} to {author_reviews['date'].max().date()}"
    }
    
    return author_reviews, summary

# Get reviews for author 202
reviews, summary = get_author_reviews(df, 202)

# Print summary statistics
print("\nSummary for Author 202:")
print(f"Total reviews: {summary['total_reviews']}")
print(f"Unique hotels reviewed: {summary['unique_hotels']}")
print(f"Average rating: {summary['average_rating']:.2f}")
print(f"Review period: {summary['date_range']}")

# Print all reviews
print("\nDetailed Reviews:")
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full text content
print(reviews[['date', 'hotel_id', 'rating', 'title', 'text']])


Summary for Author 202:
Total reviews: 1193017
Unique hotels reviewed: 38828
Average rating: 3.68
Review period: From 2001-02-01 to 2019-05-14

Detailed Reviews:
                       date  hotel_id  rating  \
3305442 2001-02-01 00:00:00    100459     2.0   
271659  2001-03-01 00:00:00      4037     5.0   
6904959 2001-04-01 00:00:00    362763     3.0   
7063305 2001-04-01 00:00:00    363206     5.0   
3463908 2001-04-01 00:00:00    110756     5.0   
...                     ...       ...     ...   
7225956 2019-05-12 23:43:21    363924     5.0   
7349739 2019-05-13 00:06:24    364821     5.0   
7310018 2019-05-13 15:57:36    364549     4.0   
7322020 2019-05-13 18:53:48    364635     4.0   
7349738 2019-05-14 00:06:23    364821     1.0   

                                                                             title  \
3305442                        a reasonable hotel near the Washington National Zoo   
271659                                                           Cozy Desert

In [13]:
def analyze_hotel_ratings(df):
    # Count total unique hotels
    total_hotels = df['hotel_id'].nunique()
    
    # Get ratings per hotel
    hotel_stats = df.groupby('hotel_id').agg({
        'rating': ['count', 'mean', 'min', 'max'],
        'author_id': 'nunique'
    }).reset_index()
    
    # Rename columns for clarity
    hotel_stats.columns = ['hotel_id', 'total_reviews', 'avg_rating', 'min_rating', 'max_rating', 'unique_reviewers']
    
    # Sort by number of reviews to see most reviewed hotels
    most_reviewed = hotel_stats.nlargest(5, 'total_reviews')
    
    # Calculate rating distribution
    rating_distribution = df['rating'].value_counts().sort_index()
    
    # Calculate summary statistics
    summary = {
        'total_hotels': total_hotels,
        'avg_reviews_per_hotel': df['hotel_id'].value_counts().mean(),
        'median_reviews_per_hotel': df['hotel_id'].value_counts().median(),
        'most_common_rating': df['rating'].mode().iloc[0],
        'overall_avg_rating': df['rating'].mean()
    }
    
    return hotel_stats, most_reviewed, rating_distribution, summary

# Run the analysis
hotel_stats, top_hotels, rating_dist, summary = analyze_hotel_ratings(df)

# Print summary statistics
print("\nHotel Rating Summary:")
print(f"Total number of unique hotels rated: {summary['total_hotels']}")
print(f"Average reviews per hotel: {summary['avg_reviews_per_hotel']:.2f}")
print(f"Median reviews per hotel: {summary['median_reviews_per_hotel']:.2f}")
print(f"Overall average rating: {summary['overall_avg_rating']:.2f}")
print(f"Most common rating: {summary['most_common_rating']}")

print("\nRating Distribution:")
print(rating_dist)

print("\nTop 5 Most Reviewed Hotels:")
print(top_hotels[['hotel_id', 'total_reviews', 'avg_rating', 'unique_reviewers']])

# Calculate hotels by review count buckets
review_count_bins = [0, 10, 50, 100, 500, 1000, float('inf')]
review_count_labels = ['1-10', '11-50', '51-100', '101-500', '501-1000', '1000+']
hotel_stats['review_bucket'] = pd.cut(hotel_stats['total_reviews'], 
                                    bins=review_count_bins, 
                                    labels=review_count_labels)
hotels_by_review_count = hotel_stats['review_bucket'].value_counts().sort_index()

print("\nHotels grouped by number of reviews:")
print(hotels_by_review_count)


Hotel Rating Summary:
Total number of unique hotels rated: 329340
Average reviews per hotel: 22.41
Median reviews per hotel: 6.00
Overall average rating: 3.98
Most common rating: 4.0

Rating Distribution:
1.0     224394
2.0     416444
3.0    1316617
4.0    2753880
5.0    2668363
Name: rating, dtype: int64

Top 5 Most Reviewed Hotels:
        hotel_id  total_reviews  avg_rating  unique_reviewers
327183    362880           2987    4.260797              1869
326122    361819           2324    4.443201               233
326916    362613           2202    4.260218               681
326681    362378           2172    4.108656               921
326827    362524           2164    4.292976               185

Hotels grouped by number of reviews:
1-10        205441
11-50        89401
51-100       20175
101-500      13562
501-1000       653
1000+          108
Name: review_bucket, dtype: int64


In [14]:
def create_formatted_hotel_analysis(df):
    # 1. Basic Hotel Statistics Table
    basic_stats = {
        'Metric': [
            'Total Unique Hotels',
            'Average Reviews per Hotel',
            'Median Reviews per Hotel',
            'Overall Average Rating',
            'Most Common Rating'
        ],
        'Value': [
            df['hotel_id'].nunique(),
            df['hotel_id'].value_counts().mean(),
            df['hotel_id'].value_counts().median(),
            df['rating'].mean(),
            df['rating'].mode().iloc[0]
        ]
    }
    basic_stats_df = pd.DataFrame(basic_stats)
    basic_stats_df['Value'] = basic_stats_df['Value'].apply(lambda x: f"{x:.2f}" if isinstance(x, float) else str(x))
    
    # 2. Rating Distribution Table
    rating_dist = df['rating'].value_counts().sort_index()
    rating_dist_df = pd.DataFrame({
        'Rating': rating_dist.index,
        'Number of Reviews': rating_dist.values,
        'Percentage': (rating_dist.values / len(df) * 100)
    })
    rating_dist_df['Percentage'] = rating_dist_df['Percentage'].apply(lambda x: f"{x:.2f}%")
    
    # 3. Most Reviewed Hotels Table
    top_hotels = df.groupby('hotel_id').agg({
        'rating': ['count', 'mean'],
        'author_id': 'nunique'
    }).reset_index()
    top_hotels.columns = ['Hotel ID', 'Total Reviews', 'Average Rating', 'Unique Reviewers']
    top_hotels = top_hotels.nlargest(10, 'Total Reviews')
    top_hotels['Average Rating'] = top_hotels['Average Rating'].apply(lambda x: f"{x:.2f}")
    
    # 4. Hotels by Review Count Range
    review_count_bins = [0, 10, 50, 100, 500, 1000, float('inf')]
    review_count_labels = ['1-10', '11-50', '51-100', '101-500', '501-1000', '1000+']
    hotel_counts = df.groupby('hotel_id').size()
    hotels_by_reviews = pd.cut(hotel_counts, bins=review_count_bins, labels=review_count_labels)
    review_range_df = pd.DataFrame({
        'Review Range': review_count_labels,
        'Number of Hotels': hotels_by_reviews.value_counts().sort_index(),
        'Percentage': (hotels_by_reviews.value_counts().sort_index() / len(hotel_counts) * 100)
    })
    review_range_df['Percentage'] = review_range_df['Percentage'].apply(lambda x: f"{x:.2f}%")
    
    # Print tables with clear headers
    print("\n=== Basic Hotel Statistics ===")
    print(basic_stats_df.to_string(index=False))
    
    print("\n=== Rating Distribution ===")
    print(rating_dist_df.to_string(index=False))
    
    print("\n=== Top 10 Most Reviewed Hotels ===")
    print(top_hotels.to_string(index=False))
    
    print("\n=== Hotels Grouped by Number of Reviews ===")
    print(review_range_df.to_string(index=False))

# Run the analysis
create_formatted_hotel_analysis(df)


=== Basic Hotel Statistics ===
                   Metric     Value
      Total Unique Hotels 329340.00
Average Reviews per Hotel     22.41
 Median Reviews per Hotel      6.00
   Overall Average Rating      3.98
       Most Common Rating      4.00

=== Rating Distribution ===
 Rating  Number of Reviews Percentage
    1.0             224394      3.04%
    2.0             416444      5.64%
    3.0            1316617     17.84%
    4.0            2753880     37.32%
    5.0            2668363     36.16%

=== Top 10 Most Reviewed Hotels ===
 Hotel ID  Total Reviews Average Rating  Unique Reviewers
   362880           2987           4.26              1869
   361819           2324           4.44               233
   362613           2202           4.26               681
   362378           2172           4.11               921
   362524           2164           4.29               185
   362084           2123           3.97               735
   362898           2050           4.56             

In [15]:
def create_formatted_author_analysis(df):
    # 1. Basic Author Statistics Table
    basic_stats = {
        'Metric': [
            'Total Unique Authors',
            'Average Reviews per Author',
            'Median Reviews per Author',
            'Average Rating Given',
            'Most Common Rating Given'
        ],
        'Value': [
            df['author_id'].nunique(),
            df['author_id'].value_counts().mean(),
            df['author_id'].value_counts().median(),
            df['rating'].mean(),
            df['rating'].mode().iloc[0]
        ]
    }
    basic_stats_df = pd.DataFrame(basic_stats)
    basic_stats_df['Value'] = basic_stats_df['Value'].apply(lambda x: f"{x:.2f}" if isinstance(x, float) else str(x))
    
    # 2. Rating Distribution by Authors Table
    rating_dist = df['rating'].value_counts().sort_index()
    rating_dist_df = pd.DataFrame({
        'Rating': rating_dist.index,
        'Number of Reviews': rating_dist.values,
        'Percentage': (rating_dist.values / len(df) * 100)
    })
    rating_dist_df['Percentage'] = rating_dist_df['Percentage'].apply(lambda x: f"{x:.2f}%")
    
    # 3. Most Active Authors Table
    top_authors = df.groupby('author_id').agg({
        'rating': ['count', 'mean'],
        'hotel_id': 'nunique'
    }).reset_index()
    top_authors.columns = ['Author ID', 'Total Reviews', 'Average Rating Given', 'Unique Hotels Reviewed']
    top_authors = top_authors.nlargest(10, 'Total Reviews')
    top_authors['Average Rating Given'] = top_authors['Average Rating Given'].apply(lambda x: f"{x:.2f}")
    
    # 4. Authors by Review Count Range
    review_count_bins = [0, 5, 10, 50, 100, 500, float('inf')]
    review_count_labels = ['1-5', '6-10', '11-50', '51-100', '101-500', '500+']
    author_counts = df.groupby('author_id').size()
    authors_by_reviews = pd.cut(author_counts, bins=review_count_bins, labels=review_count_labels)
    review_range_df = pd.DataFrame({
        'Review Range': review_count_labels,
        'Number of Authors': authors_by_reviews.value_counts().sort_index(),
        'Percentage': (authors_by_reviews.value_counts().sort_index() / len(author_counts) * 100)
    })
    review_range_df['Percentage'] = review_range_df['Percentage'].apply(lambda x: f"{x:.2f}%")
    
    # 5. Author Activity Statistics
    author_activity = df.groupby('author_id').agg({
        'hotel_id': 'nunique',
        'rating': 'count'
    }).reset_index()
    author_activity['reviews_per_hotel'] = author_activity['rating'] / author_activity['hotel_id']
    activity_stats = {
        'Metric': [
            'Average Hotels per Author',
            'Median Hotels per Author',
            'Average Reviews per Hotel per Author',
            'Max Hotels Reviewed by Single Author',
            'Max Reviews by Single Author'
        ],
        'Value': [
            author_activity['hotel_id'].mean(),
            author_activity['hotel_id'].median(),
            author_activity['reviews_per_hotel'].mean(),
            author_activity['hotel_id'].max(),
            author_activity['rating'].max()
        ]
    }
    activity_stats_df = pd.DataFrame(activity_stats)
    activity_stats_df['Value'] = activity_stats_df['Value'].apply(lambda x: f"{x:.2f}" if isinstance(x, float) else str(x))
    
    # Print all tables with clear headers
    print("\n=== Basic Author Statistics ===")
    print(basic_stats_df.to_string(index=False))
    
    print("\n=== Rating Distribution by Authors ===")
    print(rating_dist_df.to_string(index=False))
    
    print("\n=== Top 10 Most Active Authors ===")
    print(top_authors.to_string(index=False))
    
    print("\n=== Authors Grouped by Number of Reviews ===")
    print(review_range_df.to_string(index=False))
    
    print("\n=== Author Activity Statistics ===")
    print(activity_stats_df.to_string(index=False))

# Run the analysis
create_formatted_author_analysis(df)


=== Basic Author Statistics ===
                    Metric     Value
      Total Unique Authors 189992.00
Average Reviews per Author     38.84
 Median Reviews per Author     27.00
      Average Rating Given      3.98
  Most Common Rating Given      4.00

=== Rating Distribution by Authors ===
 Rating  Number of Reviews Percentage
    1.0             224394      3.04%
    2.0             416444      5.64%
    3.0            1316617     17.84%
    4.0            2753880     37.32%
    5.0            2668363     36.16%

=== Top 10 Most Active Authors ===
 Author ID  Total Reviews Average Rating Given  Unique Hotels Reviewed
       202        1193017                 3.68                   38828
      1504          23889                 4.04                   18894
       338            612                 3.83                     580
     27812            536                 3.75                     387
    194488            443                 3.81                     412
    537967     